In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
HEADER = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
url_list = [
    'https://movie.douban.com/people/98909222/wish?start=0&sort=time&rating=all&filter=all&mode=list',
    'https://movie.douban.com/people/98909222/wish?start=30&sort=time&rating=all&filter=all&mode=list'
]

In [3]:
def get_movie_info(movie_url):
    r_movie = requests.get(movie_url, headers=HEADER)
    soup_movie = BeautifulSoup(r_movie.text, 'html.parser')
    return {
        'name': str(soup_movie.find_all('span', {'property': 'v:itemreviewed'})[0].contents[0]),
        'year': int(soup_movie.find_all('span', {'class': 'year'})[0].contents[0].replace('(', '').replace(')', '')),
        'rating': float(soup_movie.find_all('strong', {'class': 'll rating_num', 'property': 'v:average'})[0].contents[0]),
        'votes': int(soup_movie.find_all('span', {'property': 'v:votes'})[0].contents[0])
    }

In [4]:
def get_page_info(url):
    r_page = requests.get(url, headers=HEADER)
    soup_page = BeautifulSoup(r_page.text, 'html.parser')  # 'lxml'
    movies = soup_page.find_all('div', {'class': 'title'})
    for i in range(0, len(movies)):
        movies_list.append(get_movie_info(movies[i].contents[1].attrs['href']))
        # print('No. ' + str(i))
    del i

In [8]:
movies_list = []
for i in range(0, len(url_list)):
    get_page_info(url_list[i])
del i

In [9]:
movies_df = pd.DataFrame(data=movies_list)
movies_df['rating_year'] = movies_df['rating'] * movies_df['year']
movies_df['rating_votes'] = movies_df['rating'] * movies_df['votes']
movies_df['rating_year_votes'] = movies_df['rating'] * movies_df['year'] * movies_df['votes']

In [10]:
movies_df.sort_values('rating_year_votes', ascending=False)

,name,year,rating,votes,rating_year,rating_votes,rating_year_votes
22,头号玩家 Ready Player One,2018,8.7,1029281,17556.6,8954744.7,1.807067e+10
47,教父 The Godfather,1972,9.3,680006,18339.6,6324055.8,1.247104e+10
46,教父2 The Godfather: Part Ⅱ,1974,9.2,376836,18160.8,3466891.2,6.843643e+09
38,社交网络 The Social Network,2010,8.1,364985,16281.0,2956378.5,5.942321e+09
1,银翼杀手2049 Blade Runner 2049,2017,8.3,345295,16741.1,2865948.5,5.780618e+09
3,雪国列车 설국열차,2013,7.5,331273,15097.5,2484547.5,5.001394e+09
44,教父3 The Godfather: Part III,1990,8.9,251388,17711.0,2237353.2,4.452333e+09
39,华尔街之狼 The Wolf of Wall Street,2013,7.9,273366,15902.7,2159591.4,4.347257e+09
34,奇幻森林 The Jungle Book,2016,7.8,218881,15724.8,1707271.8,3.441860e+09
13,角斗士 Gladiator,2000,8.5,172888,17000.0,1469548.0,2.939096e+09
